# Keras 高层接口
Keras 是一个主要由Python 语言开发的开源神经网络计算库，在TensorFlow 2 版本中，Keras被正式确定为TensorFlow 的高层唯一接口API.在TensorFlow 中，Keras 被实现在tf.keras 子模块中.

Keras 与tf.keras的区别与联系:
- Keras :安装标准的Keras 库就可以方便地调用TensorFlow、CNTK 等后端完成加速计算.
- tf.keras:它与TensorFlow 深度融合，且只能基于TensorFlow 后端运算，并对TensorFlow的支持更完美

## tf.keras 常见功能模块
### 常见网络层类
对于常见的神经网络层，可以使用张量方式的底层接口函数来实现，这些接口函数一般在`tf.nn `模块中.

更常用地，对于常见的网络层，我们一般直接使用层方式来完成模型的搭建，`tf.keras.layers`命名空间下提供了大量常见网络层的类全连接层、激活函数层、池化层、卷积层、循环神经网络层等. 对于这些网络层类，只需要在创建时指定网络层的相关参数，并调用`__call__`方法即可完成前向计算。在调用\__call__方法时，Keras 会自动调用每个层的前向传播逻辑，这些逻辑一般实现在类的call 函数中

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    print(e)

In [ ]:
# 例如创建softmax层
x = tf.constant([0.1, 0.5, 2, 1])
layer = layers.Softmax(axis=-1)
out = layer(x)  # 调用softmax前向计算
out

In [ ]:
out = tf.nn.softmax(x)  # 使用函数形式
out

### 网络容器Sequential
当网络层数变得较深时，可以通过Keras 提供的网络容器Sequential 将多个网络层封装成一个大网络模型，只需要调用网络模型的实例一次即可完成数据从第一层到最末层的顺序传播运算.

In [ ]:
network = Sequential([  # 封装为网络
    layers.Dense(3, activation=None),  # 全连接层, 不使用激活函数
    layers.ReLU(),  # 激活函数层
    layers.Dense(2, activation=None),
    layers.ReLU(),
])

In [ ]:
x = tf.random.normal([4, 3])
network(x)

In [ ]:
# add() 方法动态创建网络层
network = Sequential([])
layers_num = 2 # 堆叠2 次
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.ReLU())
network.build(input_shape=(1, 4))  # 创建网络参数 第一个batch_size任意数
network.summary()
# Trainable params 待优化的参数

In [ ]:
for params in network.trainable_variables:
    print(params.name, params.shape)  # 参数名 , shape

## 模型装配、训练与测试
在训练网络时，一般的流程是通过前向计算获得网络的输出值，再通过损失函数计算网络误差，然后通过自动求导工具计算梯度并更新，同时间隔性地测试网络的性能。对于这种常用的训练逻辑，可以直接通过Keras 提供的模型装配与训练等高层接口实现，简洁清晰.

In [ ]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

digits = load_digits()
# data = scale(digits.data)

In [ ]:
digits.images[0]

In [ ]:
def process(X, y):
    X = tf.cast(X, dtype=tf.float32) / 255.
    X = tf.reshape(X, (-1, 28*28))
    y = tf.cast(y, dtype=tf.int32)
    y_onehot = tf.one_hot(y, depth=10)
    return X, y_onehot

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.imshow(digits.images[0].reshape(8, 8)/255., cmap=plt.cm.bone)

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
# X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.3, random_state=42)
train_db = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_db = tf.data.Dataset.from_tensor_slices((X_test, y_test))
train_db = train_db.shuffle(10000).batch(512).map(process)
test_db = test_db.shuffle(10000).batch(512).map(process)

In [ ]:
X_train.shape

In [ ]:
x1, y1 = next(iter(train_db))
x1.shape, y1.shape

In [ ]:
# plt.imshow(x1[0].numpy().reshape(28, 28), cmap=plt.cm.bone)

### 模型装配
`keras.layers.Layer` 类是网络层的母类, 定义了网络层的一些常见功能，如添加权值、管理权值列表等.

`keras.Model`类是网络的母类, 除了具有Layer 类的功能，还添加了保存模型、加载模型、训练与测试模型等便捷功能。Sequential 也是Model 的子类，因此具有Model 类的所有功能。

In [ ]:
# 手写数字识别MNIST
# 创建5层网络
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu',),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])
network.build(input_shape=(None, 28*28))
network.summary()

建立网络后的训练流程:

- 迭代数据集多次 
- 每次按批产生训练数据、前向计算
- 通过损失函数计算误差值
- 反向传播自动计算梯度、更新网络参数

在Keras 中提供了compile()和fit()函数方便实现上述逻辑

In [ ]:
from tensorflow.keras import optimizers, losses

network.compile(optimizer=optimizers.Adam(lr=0.01),  # Adam优化器, 学习率0.01
               loss=losses.CategoricalCrossentropy(from_logits=True),  # 交叉熵损失,包含softmax
               metrics=['accuracy'],   # 指标为准确率
               )

### 模型训练
模型装配完成后，即可通过fit()函数送入待训练的数据集和验证用的数据集，这一步称为模型训练。

In [ ]:
history = network.fit(train_db, epochs=10,
                      validation_data=test_db,
                      validation_freq=2)  # 每2个epoch验证一次

In [ ]:
history.history

### 模型测试
Model 基类除了可以便捷地完成网络的装配与训练、验证，还可以非常方便的预测和测试.

In [ ]:
x, y = next(iter(test_db))  # 加载一个batch数据

In [ ]:
y.shape

In [ ]:
out = network.predict(x)  # 模型预测，预测结果保存在out 中
y_pred = np.argmax(out, axis=-1)  

In [ ]:
out.shape

In [ ]:
y_t = np.argmax(y, axis=-1)

In [ ]:
np.sum(y_t.ravel() == y_pred.ravel()) / len(y_pred)

In [ ]:
network.evaluate(test_db)  # test_db上的性能测试

## 模型的保存和加载
### 张量方式

In [ ]:
network.save_weights('mnist_weights.ckpt')

In [ ]:
del network

In [ ]:
network = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])
network.compile(optimizer=optimizers.Adam(lr=0.01),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

In [ ]:
network.load_weights('mnist_weights.ckpt')

In [ ]:
network.build(input_shape=(4, 8*8))
network.summary()

这种保存与加载网络的方式最为轻量级，文件中保存的仅仅是张量参数的数值，并没有其
它额外的结构参数。但是它需要使用相同的网络结构才能够正确恢复网络状态，因此一般
在拥有网络源文件的情况下使用

### 网络方式

model.h5 文件除了保存了模型参数外，还应保存了网络结构信息，不需要提前
创建模型即可直接从文件中恢复出网络network 对象

In [ ]:
network.save('model.h5')
del network

In [ ]:
network = keras.models.load_model('model.h5')
network.build(input_shape=(4, 8*8))
network.summary()

### SavedModel
SavedModel 方式具有平台无关性

In [ ]:
tf.saved_model.save(network, 'model-savedmodel')

In [ ]:
network = tf.saved_model.load('model-savedmodel')
network

In [ ]:
tf.saved_model.load?

In [ ]:
acc_meter = keras.metrics.CategoricalAccuracy()
for x, y in test_db:
    pred = network(x)
    acc_meter.update_state(y_true=y, y_pred=pred)
print("Test Accuracy:%f" % acc_meter.result())

## 自定义网络
### 自定义网络层

至少需要实现初始化\__init__方法和前向传播逻辑call 方法

In [ ]:
layers.Layer.add_weight

In [ ]:
layers.Layer?

In [ ]:
class MyDense(layers.Layer):
    def __init__(self, input_dim, out_dim):
        super().__init__()
        # 创建权值张量并添加到类管理列表中，设置为需要优化
        self.kernel = self.add_weight(shape=(input_dim, out_dim), trainable=True)
        # 没有偏置
    def call(self, inputs, training=None):
        # 实现前向计算
        out = inputs @ self.kernel
        out = tf.nn.relu(out)
        return out

In [ ]:
net = MyDense(4, 3)
net.weights, net.trainable_variables
# 类初始化中创建为tf.Variable类型的类成员变量也会自动纳入张量管理中

In [ ]:
net.variables

In [ ]:
x = tf.random.normal([3, 4])
x

In [ ]:
net(x)

### 自定义网络

Sequential 容器适合于数据按序从第一层传播到第二层，再从第二层传播到第三层，以此规律传播的网络模型.

对于复杂的网络结构, 使用自定义网络类更加灵活

In [ ]:
network1 = Sequential([
    MyDense(784, 256), 
    MyDense(256, 128), 
    MyDense(128, 64), 
    MyDense(64, 32), 
    MyDense(32, 10), 
])
network1.build(input_shape=(None, 28*28))
network1.summary()

In [ ]:
class MyModel(keras.Model):# 自定义网络类，继承自 Model 基类
    def __init__(self):
        super().__init__()
        self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
    def call(self, inputs, training=None):
        # 自定义前向运算逻辑
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x

In [ ]:
my_model = MyModel()
my_model.build(input_shape=(None, 28*28))
my_model.summary()

## 模型乐园
对于常用的网络模型，如ResNet、VGG 等，不需要手动创建网络，可以直接从keras.applications 子模块中创建, 同时还可以通过设置weights 参数加载预训练的网络参数，非常方便.

In [ ]:
resnet = keras.applications.ResNet50(weights='imagenet', include_top=False)

## 测量工具
Keras 提供了一些常用的测量工具，位于keras.metrics 模块中，专门用于统计训练过程中常用的指标数据.
Keras 的测量工具的使用方法一般有4 个主要步骤：
1. 新建测量器;
2. 写入数据;
3. 读取统计数据;
4. 清零测量器。

In [ ]:
# 新建测量器
loss_meter = keras.metrics.Mean()

In [ ]:
loss = tf.random.normal((1, 32), seed=42)
loss

In [ ]:
# 写入数据
loss_meter.update_state(loss)

In [ ]:
# 读取统计信息
# 在采样多次数据后，可以选择在需要的地方调用测量器的result()函数，来获取统计值
loss_meter.result()

In [ ]:
# 由于测量器会统计所有历史记录的数据，因此在启动新一轮统计时，有必要清除历史状态
loss_meter.reset_states()

In [ ]:
loss_meter.result()

**计算准确率**

In [ ]:
acc_meter = keras.metrics.Accuracy()
# Accuracy 类的update_state函数的参数为预测值和真实值
x, y_true = next(iter(test_db))
out = network(x)  # (b, 10)
y_pred = tf.argmax(out, axis=-1)
y_pred

In [ ]:
y_true = tf.argmax(y_true, axis=-1)
y_true

In [ ]:
acc_meter.update_state(y_true, y_pred)

In [ ]:
acc_meter.result().numpy()

In [ ]:
# 清零测量器
acc_meter.reset_states()

## 可视化
---
TensorFlow 提供了一个专门的可视化工具，叫做TensorBoard，它通过TensorFlow 将监控数据写入到文件系统，并利用Web后端监控对应的文件目录，从而可以允许用户从远程查看网络的监控数据.
### 模型端
在模型端，需要创建写入监控数据的Summary 类，并在需要的时候写入监控数据.

In [ ]:
log_dir = r'D:/tensorflow_data'
# 创建监控类，监控数据将写入log_dir 目录
summary_writer = tf.summary.create_file_writer(log_dir)
acc_meter = keras.metrics.Accuracy()

In [ ]:
train_db = train_db.repeat(20)

In [ ]:
optimizer = optimizers.Adam(lr=0.01)
for step, (x, y) in enumerate(train_db):
    with tf.GradientTape() as tape:
        out = my_model(x)
        loss = losses.categorical_crossentropy(y, out, from_logits=True)
    grads = tape.gradient(loss, my_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, my_model.trainable_variables))
    if step % 118 == 0:
        # print(tf.reduce_mean(loss).numpy())
        for x, y in test_db:
            pred_out = my_model(x)
            y_pred = tf.argmax(pred_out, axis=-1)
            y_true = tf.argmax(y, axis=-1)
            acc_meter.update_state(y_true, y_pred)
        # print(acc_meter.result().numpy())
       
        with summary_writer.as_default():  # 写入环境
            # 当前时间戳step 上的数据为loss，写入到名为train-loss 数据库中
            tf.summary.scalar('train-loss', float(loss.numpy().mean()), step=step)  # step 参数类似于每个数据对应的时间刻度信息
            tf.summary.scalar('test-acc', float(acc_meter.result()), step=step)
            # 可视化真实标签的直方图分布
            tf.summary.histogram('y-hist',y_true, step=step)
            # 查看文本信息
            tf.summary.text('loss-text',str(float(loss.numpy().mean())), step=step)
            # 可视化测试用的图片，设置最多可视化9 张图片
            # tf.summary.image("val-onebyone-images:", val_images, max_outputs=9, step=step)
        acc_meter.reset_states()

### 浏览器端

在cmd 终端运行`tensorboard --logdir path `指定Web 后端监控的文件目录path，即可打开Web 后端监控进程:
```shell
tensorboard --logdir d:\tensorflow_data
```